In [59]:
import glob
from collections import defaultdict
from pprint import pprint
import pickle
import operator
import os
import nltk
import sklearn
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams
import random
import re
import pandas as pd
import json

In [6]:
riloff_eval_dataset = pickle.load(open('march22/sarcasm-annos-emnlp13-tweet_objs-2124.pkl','rb'))

In [7]:
all_ids = set([tweet['id'] for tweet in riloff_eval_dataset])

In [10]:
riloff_eval_dataset_alt = pd.read_csv('march22/riloff-tweets.orig',delimiter='\t',names=['tweet id','label','tweet'])

In [15]:
all_ids_alt = set(riloff_eval_dataset_alt['tweet id'])

In [22]:
riloff_orig_dataset = pd.read_csv('march22/sarcasm-annos-emnlp13.tsv',delimiter='\t',names=['tweet id','label'])

In [25]:
#all_ids_orig = set(riloff_orig_dataset['tweet id'])
len(all_ids_orig - all_ids)

876

In [27]:
riloff_orig_dataset2 = riloff_orig_dataset.set_index('tweet id')

In [51]:
riloff_orig_dataset2 = {}
for twid,label in zip(riloff_orig_dataset['tweet id'],riloff_orig_dataset['label']):
    riloff_orig_dataset2[twid] = label

In [56]:
new_eval_dataset = []
for twobj in riloff_eval_dataset:
    d = {'id':twobj['id'],'text':twobj['text'],'label':riloff_orig_dataset2[twobj['id']]}
    new_eval_dataset.append(d)

In [ ]:
count = 0
for d in new_eval_dataset:
    if '\n' in d['text']:
        print(d['text'])
        print('###')
        count += 1

print("\n\n\ncount",count)

In [62]:
with open('/home/nikhil/Desktop/245/Project/code/ark-tweet-nlp-0.3.2/riloff-orig.json','w') as fw:
    for d in new_eval_dataset:
        fw.write(json.dumps(d)+'\n')

In [71]:
i = 0
tweets[i].update({'tokens':[],'tags':[],'conf':[]})
with open('/home/nikhil/Desktop/245/Project/code/ark-tweet-nlp-0.3.2/riloff-orig.tagger.out.conll') as f:
    for line in f:
        if line.strip() == '':
            i += 1
            tweets[i].update({'tokens':[],'tags':[],'conf':[]})
        else:
            word,tag,conf = line.strip().split()
            tweets[i]['tokens'].append(word)
            tweets[i]['tags'].append(tag)
            tweets[i]['conf'].append(conf)

In [75]:
pickle.dump(tweets,open('march27/riloff-tokenized-and-tagged.pkl','wb'))

In [78]:


def remove_trailing_nums(text):
    return re.sub("\d+$", "", text)

contents = {}
tweets = []
alreadyseen = set()

for filename in glob.iglob('data/search-api/22ndMarch2016/*.pkl'):
    contents[filename] = pickle.load(open(filename,'rb'))
    if type(contents[filename][0]) == tuple:
        for tup in contents[filename]:
            tw_id, tweet = tup
            if tw_id not in alreadyseen:
                alreadyseen.add(tw_id)
                tweets.append({'label':'SARCASM','text':tweet})
    elif type(contents[filename][0]) == dict:
        for d in contents[filename]:
            if d['id'] not in alreadyseen:
                alreadyseen.add(d['id'])
                temp = {'id':d['id'],'label':'SARCASM','text':d['text']}
                fn = os.path.basename(filename)
                if fn[0] == '#':
                    temp['#type'] = fn[:-4]
                tweets.append(temp)
                    
for filename in glob.iglob('data/search-api/5thMarch2016/*.pkl'):
    contents[filename] = pickle.load(open(filename,'rb'))
    if type(contents[filename][0]) == tuple:
        for tup in contents[filename]:
            tw_id, tweet = tup
            if tw_id not in alreadyseen:
                alreadyseen.add(tw_id)
                tweets.append({'label':'SARCASM','text':tweet})
    elif type(contents[filename][0]) == dict:
        for d in contents[filename]:
            if d['id'] not in alreadyseen:
                alreadyseen.add(d['id'])
                temp = {'id':d['id'],'label':'SARCASM','text':d['text']}
                fn = os.path.basename(filename)
                if 'sarcasm+yeahright+justkidding-5thMarch-' not in fn:
                    temp['#type'] = '#'+remove_trailing_nums(os.path.basename(filename)[:-4])
                tweets.append(temp)
                    
for filename in glob.iglob('data/scraped/*.pkl'):
    contents[filename] = pickle.load(open(filename,'rb'))
    for d in contents[filename]:
        if d['id'] not in alreadyseen:
            alreadyseen.add(d['id'])
            if type(d['text']) == bytes:
                d['text'] = str(d['text'],'utf-8')
            temp = {'id':d['id'],'label':'SARCASM','text':d['text']}
            fn = os.path.basename(filename)
            if fn == 'scraped-with-JUSTKIDDING-until-30Jun2015.pkl':
                temp['#type'] = '#justkidding'
            elif fn == 'scraped-with-NOT-until-28Dec2015.pkl':
                temp['#type'] = '#not'
            tweets.append(temp)

In [80]:
with open('/home/nikhil/Desktop/245/Project/code/ark-tweet-nlp-0.3.2/scraped-orig.json','w') as fw:
    for d in tweets:
        fw.write(json.dumps(d)+'\n')

In [ ]:
#with open('/home/nikhil/Desktop/245/Project/code/ark-tweet-nlp-0.3.2/scraped-orig-noretweets.json','w') as fw:
    for d in tweets:
        if d['text'][:3] != 'RT '
        fw.write(json.dumps(d)+'\n')

In [83]:
tweets.append({'appeasing_dummy':None})
            
i = 0
tweets[i].update({'tokens':[],'tags':[],'conf':[]})
with open('/home/nikhil/Desktop/245/Project/code/ark-tweet-nlp-0.3.2/scraped-orig.tagged') as f:
    for line in f:
        if line.strip() == '':
            i += 1
            tweets[i].update({'tokens':[],'tags':[],'conf':[]})
        else:
            word,tag,conf = line.strip().split()
            tweets[i]['tokens'].append(word)
            tweets[i]['tags'].append(tag)
            tweets[i]['conf'].append(conf)

In [91]:
pickle.dump(tweets,open('march27/scraped-sarc-tokenized-and-tagged.pkl','wb'))

In [92]:
tweets_nort = [tweet for tweet in tweets if tweet['text'][:3]!='RT ']

In [96]:
len(tweets_nort)

65486

In [97]:
tweets = tweets_nort

In [98]:
tweets.append({'appeasing_dummy':None})
            
i = 0
tweets[i].update({'tokens':[],'tags':[],'conf':[]})
with open('/home/nikhil/Desktop/245/Project/code/ark-tweet-nlp-0.3.2/scraped-orig-noretweets.tagged') as f:
    for line in f:
        if line.strip() == '':
            i += 1
            tweets[i].update({'tokens':[],'tags':[],'conf':[]})
        else:
            word,tag,conf = line.strip().split()
            tweets[i]['tokens'].append(word)
            tweets[i]['tags'].append(tag)
            tweets[i]['conf'].append(conf)

In [102]:
pickle.dump(tweets,open('march27/scraped-sarc-noretweets-tokenized-and-tagged.pkl','wb'))

In [103]:
len(tweets)

65486

In [104]:
ls march27

riloff-tokenized-and-tagged.pkl
scraped-sarc-noretweets-tokenized-and-tagged.pkl
scraped-sarc-tokenized-and-tagged.pkl


In [107]:
from sklearn.metrics import classification_report
y_true = [0, 1, 2, 2, 0]
y_pred = [0, 0, 2, 1, 0]
print(classification_report(y_true, y_pred))

             precision    recall  f1-score   support

          0       0.67      1.00      0.80         2
          1       0.00      0.00      0.00         1
          2       1.00      0.50      0.67         2

avg / total       0.67      0.60      0.59         5

